In [62]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from patsy import dmatrices
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score


In [4]:
# load dataset
dta = sm.datasets.fair.load_pandas().data

# add "affair" column: 1 represents having affairs, 0 represents not
dta['affair'] = (dta.affairs > 0).astype(int)

In [6]:
#To check the column names
dta.columns

Index(['rate_marriage', 'age', 'yrs_married', 'children', 'religious', 'educ',
       'occupation', 'occupation_husb', 'affairs', 'affair'],
      dtype='object')

In [13]:
## To Prepare Data for Logistic Regression
y, X = dmatrices('affair ~ rate_marriage + age + yrs_married + children + \
                  religious + educ + C(occupation) + C(occupation_husb)',
                  dta, return_type="dataframe")

In [17]:
X.columns

Index(['Intercept', 'C(occupation)[T.2.0]', 'C(occupation)[T.3.0]',
       'C(occupation)[T.4.0]', 'C(occupation)[T.5.0]', 'C(occupation)[T.6.0]',
       'C(occupation_husb)[T.2.0]', 'C(occupation_husb)[T.3.0]',
       'C(occupation_husb)[T.4.0]', 'C(occupation_husb)[T.5.0]',
       'C(occupation_husb)[T.6.0]', 'rate_marriage', 'age', 'yrs_married',
       'children', 'religious', 'educ'],
      dtype='object')

In [18]:
# to rename column names of X

X = X.rename(columns = {'C(occupation)[T.2.0]': 'occ_2', 
                        'C(occupation)[T.3.0]':'occ_3',
                        'C(occupation)[T.4.0]':'occ_4',
                        'C(occupation)[T.5.0]':'occ_5',
                        'C(occupation)[T.6.0]':'occ_6',
                        'C(occupation_husb)[T.2.0]':'occ_husb_2',
                        'C(occupation_husb)[T.3.0]':'occ_husb_3',
                        'C(occupation_husb)[T.4.0]':'occ_husb_4',
                        'C(occupation_husb)[T.5.0]':'occ_husb_5',
                        'C(occupation_husb)[T.6.0]':'occ_husb_6'})

In [19]:
# We will have to flatten y into a 1-D array for scikit-learn 
y = np.ravel(y)

Logistic Regression

In [27]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = md.fit(X, y)

In [28]:
# To check the accuracy on the training set
model.score(X, y)

0.7258875274897895

In [29]:
# To find what percentage had affairs
y.mean()

0.3224945020420987

i.e, 32% women had affairs, which means, we could obtain 68% (100-32 = 68) accuracy.
Let's now examine the coefficients.

In [31]:
# To examine the coefficients
pd.DataFrame(list(zip(X.columns, np.transpose(model.coef_))))

,0,1
0,Intercept,[1.489835891324933]
1,occ_2,[0.18806639024440983]
2,occ_3,[0.4989478668156914]
3,occ_4,[0.25066856498524825]
4,occ_5,[0.8390080648117001]
5,occ_6,[0.8339084337443315]
6,occ_husb_2,[0.1906359445867889]
7,occ_husb_3,[0.2978327129263421]
8,occ_husb_4,[0.1614088540760616]
9,occ_husb_5,[0.18777091388972483]


Observation:

1. Marriage rating and religiousness has negative impact on the likelihood of having an affair, i.e, an increase in marriage rating and religiousness will decrease the likelihood of having an affair.

2. Lowest likelihood of having an affair corresponds to the baseline occupation (student).

In [38]:
# Model Evaluation Using a Validation Set
# To evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)
nu_model = LogisticRegression()
nu_model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [40]:
# To predict class labels for the test set
pred = nu_model.predict(X_test)

In [41]:
pred

array([1., 0., 0., ..., 1., 0., 0.])

In [42]:
# To generate class probabilities
probability = nu_model.predict_proba(X_test)


In [43]:
probability

array([[0.35971898, 0.64028102],
       [0.9113173 , 0.0886827 ],
       [0.72671132, 0.27328868],
       ...,
       [0.35251605, 0.64748395],
       [0.6107136 , 0.3892864 ],
       [0.62412678, 0.37587322]])

    The classifier is predicting 1, i.e, having an affair, anytime the probability in the second column is >0.5

In [45]:
# To generate evaluation metrics
print(metrics.accuracy_score(y_test, pred))
print(metrics.roc_auc_score(y_test, pred))

0.7306044740599714
0.6400771861340844


    Accuracy is 73%, which is similar to training and predicting on the same data.

In [48]:
#******************Model Evaluation Using Cross-Validation***********************

# To evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)

In [49]:
print(scores)
print(scores.mean())

[0.72100313 0.70219436 0.73824451 0.70597484 0.70597484 0.72955975
 0.7327044  0.70440252 0.75157233 0.75      ]
0.7241630685514876


    It is still performing at 73% accuracy.

In [ ]:
#*******************Predicting the Probability of an Affair********************
#Let's predict the probability of an affair for a random woman not present in the dataset. 
##Assumption: 29-year-old housewife who graduated college, has been married for 5 years, has 2 kids, rates herself as strongly religious, rates her marriage as fair, and her husband is a farmer.

In [61]:
model.predict_proba(np.array([[1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 5, 29, 3, 2, 4, 16]]))

array([[0.94573941, 0.05426059]])

     Predicted probability of an affair is 5%